# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

# Setup

In [211]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()
from django.conf import settings
#set debug to true
settings.DEBUG = True

#logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## Data

Import data dari model yang telah dibuat sebelumnya.

```python

In [212]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter

Ekstrak data dari model yang telah dibuat sebelumnya.

```python

In [213]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories.values()
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants.values()
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules.values()
    
class ModuleData:

    def __init__(self, module = Module):
        self._module = module
        self._modules = module.objects.all()

    @property
    def modules(self): 
        return self._modules.values()
    
    def get_module(self,id):
        module = self._modules.filter(id=id).first()
        return module
    
    def get_module_chapter(self, id):
        module = self.get_module(id)
        return module.chapters.all().values()
    
    def get_group(self, id):
        module = self.get_module(id)
        return module.groups.all().values()
    
    def get_laboratory(self, id):
        module = self.get_module(id)
        return module.laboratory
    
        
class ChapterData:

    def __init__(self, chapter = Chapter):
        self._chapter = chapter
        self._chapters = chapter.objects.all()

    @property
    def chapters(self): 
        return self._chapters.values()
    
    def get_chapter(self,id):
        chapter = self._chapters.filter(id=id).first()
        return chapter
    
    def get_chapter_module(self, id):
        chapter = self.get_chapter(id)
        return chapter.module

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants.values()
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups.values()
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def get_module_group(self, module_id):
        groups = self._groups.filter(module_id=module_id)
        return groups.values()
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants.values()

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules

Constants

```python

In [214]:
# Constant.py
class Constant:
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

# Algoritma Genetika

## Gene Representation

```python

In [215]:
from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

class Gene:
    def __init__(self, laboratory, module, module_chapter, group, assistant, time_slot: TimeSlot):
        self.laboratory = laboratory
        self.module = module
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Gene(lab={self.laboratory}, module={self.module}, module_chapter={self.module_chapter}, group={self.group}, assistant={self.assistant}, time_slot={self.time_slot})"

Test Gene Representation

```python

In [216]:
# Test Gene

# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()
chapter_data = ChapterData()

gene = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[0]["id"],assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift1"))
gene2 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[1]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift2"))
gene3 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[2]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Monday", "Shift1"))

### Gene Level Constraints

```python

In [217]:
# Constraint Class for Constraint Function
# 1. Ensure that each chapter is assigned to the correct module.
# 2. Ensure that each module is assigned to the correct lab.
# 3. Ensure that each group is assigned to the correct module.
# 4. Ensure that each assistant is assigned to the correct lab.

# import LaboratoryData, ModuleData, GroupData, AssistantData, ChapterData

class ConstraintChecker:
    def __init__(self, gene, **kwargs):
        kwargs.setdefault("lab_data", LaboratoryData())
        kwargs.setdefault("module_data", ModuleData())
        kwargs.setdefault("group_data", GroupData())
        kwargs.setdefault("assistant_data", AssistantData())
        kwargs.setdefault("chapter_data", ChapterData())
        self.gene = gene
        self.lab_data = lab_data
        self.module_data = module_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data

    def chapter_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        chapter = self.chapter_data.get_chapter(self.gene.module_chapter)
        return module == chapter.module
    
    def module_lab_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        return module.laboratory.id == self.gene.laboratory
    
    def group_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        group = self.group_data.get_group(self.gene.group)
        return module == group.module
    
    def assistant_lab_constraint(self):
        assistant = self.assistant_data.get_assistant(self.gene.assistant)
        return assistant.laboratory.id == self.gene.laboratory
    
    def schedule_constraint(self):
        return group_data.get_group_schedule(self.gene.group)[self.gene.time_slot.day][self.gene.time_slot.shift]
    
    def check(self):
        return self.chapter_module_constraint() and self.module_lab_constraint() and self.group_module_constraint() and self.assistant_lab_constraint()

Test Gene Level Constraints

```python

In [218]:
# Test ConstraintChecker
ConstraintChecker(gene).check()

True

## Chromosome Representation

```python

In [219]:
# Chromosome.py
# Chromosome Representation
# 1. Gene
# 2. Fitness

class Chromosome:
    def __init__(self, genes):
        self.genes = genes
        self.fitness = None

    def __repr__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness})"
    
    def __str__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness})"
    

### Chromosome Level Constraints

```python

Objective class
1. Minimize conflict: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.
2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

#### Objective Class for Fitness Function
1. Minimize Conflicts: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

In [220]:
#minimize_conflicts.py
from collections import namedtuple

class MinimizeConflicts:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.conflicts = namedtuple("Conflicts", ["time_slot", "lab"])
        self.conflicts_time_slot = set()
        self.assigned_time_slot = set()
        self.conflict_count = 0


    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Conflicts Time Slot: {self.conflicts_time_slot}, Assigned Time Slot: {self.assigned_time_slot}, Conflict Count: {self.conflict_count}"
    
    def __eq__(self, other):
        return self.chromosome == other.chromosome and self.conflicts_time_slot == other.conflicts_time_slot and self.assigned_time_slot == other.assigned_time_slot and self.conflict_count == other.conflict_count
    
    def calculate_conflicts(self):
        for gene in self.chromosome.genes:
            if (gene.laboratory, gene.time_slot) in self.assigned_time_slot:
                self.conflicts_time_slot.add(self.conflicts(gene.time_slot, gene.laboratory))
                self.conflict_count += 1
            else:
                self.assigned_time_slot.add((gene.laboratory, gene.time_slot))
        return self.conflict_count

    def calculate_fitness(self):
        if settings.DEBUG:
            logger.info(f"Calculating conflicted group time slots for chromosome {self.chromosome}")
        self.calculate_conflicts()
        return self.conflict_count

2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.


In [221]:
from collections import defaultdict

class MaximizeResourceUtilization:
    def __init__(self, chromosome:list, max_groups_per_assistant, max_shift_per_assistant):
        self.chromosome = chromosome
        self.max_groups_per_assistant = max_groups_per_assistant
        self.max_shift_per_assistant = max_shift_per_assistant
        self.assistants_assignments = defaultdict(list)
        self.assistants_shifts = defaultdict(list)
        self.overload_penalty = 0

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Max Groups Per Assistant: {self.max_groups_per_assistant}, Max Shift Per Assistant: {self.max_shift_per_assistant}, Assistants Assignments: {self.assistants_assignments}, Assistants Shifts: {self.assistants_shifts}, Overload Penalty: {self.overload_penalty}"
    
    def calculate_assistants_assignments(self):
        self.assistants_assignments.clear()
        for gene in self.chromosome.genes:
            self.assistants_assignments[gene.assistant].append(gene.group)
        return self.assistants_assignments
    
    def calculate_assistants_shifts(self):
        self.assistants_shifts.clear()
        for gene in self.chromosome.genes:
            self.assistants_shifts[gene.assistant].append(gene.time_slot)
        return self.assistants_shifts
    
    def calculate_overload_penalty(self):
        self.overload_penalty = 0
        self.calculate_assistants_assignments()
        self.calculate_assistants_shifts()
        for assistant , assignments in self.assistants_assignments.items():
            assignments_penalty = max(0, len(assignments) - self.max_groups_per_assistant)
            shift_penalty = (max(0, len(self.assistants_shifts[assistant]) - self.max_shift_per_assistant))
            self.overload_penalty += assignments_penalty + shift_penalty
        return self.overload_penalty
    
    def calculate_fitness(self):
        if settings.DEBUG:
            logger.info(f"Calculating overload penalty for assistant distribution: {self.chromosome}")
        self.calculate_overload_penalty()
        return self.overload_penalty

3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.


In [222]:
from collections import defaultdict

class ParticipantAvailability:
    def __init__(self, chromosome, group_data: GroupData):
        self.chromosome = chromosome
        self.group_data = group_data
        self.unsatified_participants = defaultdict(list)
        self.unsatified_participants_count = 0

    def calculate_unsatisfied_participants(self):
        self.unsatified_participants.clear()
        for gene in self.chromosome.genes:
            participant_availability = group_data.get_group_schedule(gene.group)
            if not participant_availability[gene.time_slot.day][gene.time_slot.shift]:
                self.unsatified_participants[gene.group].append(gene.time_slot)
                self.unsatified_participants_count += 1
        return self.unsatified_participants_count
    
    def calculate_fitness(self):
        if settings.DEBUG:
            logger.info(f"Calculating unsatisfied participants schedule for chromosome {self.chromosome}")
        self.calculate_unsatisfied_participants()
        return self.unsatified_participants_count

# Population Initialization

In [223]:
# import fitness_function

class Population:
    def __init__(self, chromosome, fitness_function):
        self.chromosome = chromosome
        self.fitness_function = fitness_function

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def __str__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def calculate_fitness(self):
        for chromosome in self.chromosome:
            chromosome.fitness = self.fitness_function(chromosome).calculate_fitness()


# Helper Functions

## Factory Method

```python

In [224]:
# Factory.py
from random import choice, randint
from math import ceil
from collections import namedtuple
from datetime import timedelta
# import LaboratoryData, ModuleData, GroupData, AssistantData, ChapterData
# import Constant, ConstraintChecker, Gene, Chromosome, Population

class Factory:
    def __init__(self, **kwargs):
        
        self.lab_data = kwargs.get("lab_data", LaboratoryData())
        self.module_data = kwargs.get("module_data", ModuleData())
        self.group_data = kwargs.get("group_data", GroupData())
        self.assistant_data = kwargs.get("assistant_data", AssistantData())
        self.chapter_data = kwargs.get("chapter_data", ChapterData())

    def create_time_slot(self, module_id):
        start = self.module_data.get_module(module_id).start_date
        end = self.module_data.get_module(module_id).end_date
        duration = (end - start).days + 1
        weeks_duration = ceil(duration / 7)
        week = randint(0, weeks_duration)
        day = choice(Constant.days)
        shift = choice(Constant.shifts)
        date = start + timedelta(days=week*7 + Constant.days.index(day))
        return TimeSlot(date, day, shift)

    def generate_gene(self):
        while True:
            laboratory = choice(self.lab_data.laboratories)["id"]
            module = choice(self.lab_data.get_module(laboratory))["id"]
            module_chapter = choice(self.module_data.get_module_chapter(module))["id"]
            group = choice(self.module_data.get_group(module))["id"]
            assistant = choice(self.lab_data.get_assistant(laboratory))["id"]

            time_slot = self.create_time_slot(module_id=module)

            gene = Gene(laboratory, module, module_chapter, group, assistant, time_slot)
            constraint_checker = ConstraintChecker(gene)
            if constraint_checker.check():
                return gene

    def calculate_chromosome_length(self):
        chromosome_size = 0
        for module in self.module_data.modules:
            module_chapter_count = self.module_data.get_module_chapter(module["id"]).count()
            group_count = self.module_data.get_group(module["id"]).count()
            chromosome_size += module_chapter_count * group_count
        return chromosome_size
    
    def generate_chromosome(self):
        if settings.DEBUG:
            logger.info("Generating chromosome...")
        chromosome_length = self.calculate_chromosome_length()
        genes = []
        for _ in range(chromosome_length):
            gene = self.generate_gene()
            genes.append(gene)
        return Chromosome(genes)

    def generate_population(self, population_size):
        if settings.DEBUG:
            logger.info("Generating population...")
        chromosome = []
        for _ in range(population_size):
            chromosome.append(self.generate_chromosome())
        return Population(chromosome, None)

In [225]:
#test Factory
factory = Factory()
factory.generate_population(1)

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...


Chromosome: [Chromosome(genes=[Gene(lab=1, module=1, module_chapter=6, group=22, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 19, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Saturday', shift='Shift6')), Gene(lab=1, module=1, module_chapter=5, group=6, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 15, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Tuesday', shift='Shift1')), Gene(lab=1, module=1, module_chapter=1, group=2, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 11, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift3')), Gene(lab=1, module=1, module_chapter=8, group=28, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 14, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift6')), Gene(lab=1, module=1, module_chapter=1, group=12, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 14, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), 

## Fitness Function

```python

In [226]:
class FitnessFunction:
    def __init__(self, chromosome, all: bool = False):
        self.chromosome = chromosome
        self.fitness_function = [MinimizeConflicts(chromosome), MaximizeResourceUtilization(chromosome, 2, 2), ParticipantAvailability(chromosome, GroupData())]

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def __str__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def calculate_all_fitness(self):
        fitness = []
        for fitness_function in self.fitness_function:
            fitness.append(fitness_function.calculate_fitness())
        return fitness
    
    def calculate_fitness(self):
        fitness = self.calculate_all_fitness()
        return sum(fitness)

In [227]:
#test FitnessFunction
factory = Factory()
population = factory.generate_population(2)
fitness_function = FitnessFunction
population.fitness_function = fitness_function

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...
INFO:__main__:Generating chromosome...


In [231]:
population.calculate_fitness()

INFO:__main__:Calculating conflicted group time slots for chromosome Chromosome(genes=[Gene(lab=1, module=1, module_chapter=3, group=27, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 25, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Tuesday', shift='Shift5')), Gene(lab=1, module=1, module_chapter=8, group=17, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 28, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Monday', shift='Shift4')), Gene(lab=1, module=1, module_chapter=1, group=30, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 31, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Monday', shift='Shift4')), Gene(lab=1, module=1, module_chapter=4, group=36, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 1, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Tuesday', shift='Shift1')), Gene(lab=1, module=1, module_chapter=2, group=27, assistant=6, time_slot=TimeSlot(date=datetime.datetime(2022, 1

In [232]:
population.chromosome[1].fitness

824

## Operator

```python

### Genetic Operators

```python

#### Mutation

```python

In [103]:
from copy import deepcopy
from random import random, randint, choice

class Mutation:
    def __init__(self, parent, **kwargs):
        self.parent = parent
        self.lab_data = kwargs.get("lab_data", LaboratoryData())
        self.module_data = kwargs.get("module_data", ModuleData())
        self.group_data = kwargs.get("group_data", GroupData())
        self.assistant_data = kwargs.get("assistant_data", AssistantData())
        self.chapter_data = kwargs.get("chapter_data", ChapterData())
        self.factory = kwargs.get("factory", Factory())

        self.mutation_rate = kwargs.get("mutation_rate", 0.5)
        self.mutation_size = kwargs.get("mutation_size", 1)


    def mutate(self):
        if settings.DEBUG:
            logger.info("Mutating...")
        if random() < self.mutation_rate:
            chromosome = deepcopy(self.parent)
            for _ in range(self.mutation_size):
                chromosome[self.mutation_point()] = self.factory.generate_gene()
            return chromosome
        
    def mutation_point(self):
        return randint(0, len(self.parent) - 1)
        
    
#test Mutation
factory = Factory()
population = factory.generate_population(1)
population.chromosome[0].genes
mutation = Mutation(population.chromosome[0].genes)
a = mutation.mutate()

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...
INFO:__main__:Mutating...


#### Cross Over

```python

In [104]:
class Crossover:
    def __init__(self, parent1, parent2, **kwargs):
        self.parent1 = parent1
        self.parent2 = parent2

        self.crossover_probability = kwargs.get("crossover_probability", 0.5)
        self.number_of_crossover_points = kwargs.get("number_of_crossover_points", 1)
        

    def crossover(self):
        if settings.DEBUG:
            logger.info("Crossover...")

        if random() > self.crossover_probability:
            return self.parent1, self.parent2

        chromosome1 = deepcopy(self.parent1)
        chromosome2 = deepcopy(self.parent2)

        for _ in range(self.number_of_crossover_points):
            point = self.crossover_point()
            chromosome1[point:], chromosome2[point:] = chromosome2[point:], chromosome1[point:]

        return chromosome1, chromosome2
    
    def crossover_point(self):
        return randint(0, len(self.parent1) - 1)
    
#test Crossover
factory = Factory()
population = factory.generate_population(2)
population.chromosome[0].genes
population.chromosome[1].genes
crossover = Crossover(population.chromosome[0].genes, population.chromosome[1].genes)
a, b = crossover.crossover()
a, b


        

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...
INFO:__main__:Generating chromosome...
INFO:__main__:Crossover...


([Gene(lab=1, module=1, module_chapter=4, group=21, assistant=4, time_slot=TimeSlot(date=datetime.datetime(2022, 12, 2, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift1')),
  Gene(lab=1, module=1, module_chapter=5, group=23, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 7, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Monday', shift='Shift2')),
  Gene(lab=1, module=1, module_chapter=4, group=32, assistant=2, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 16, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift4')),
  Gene(lab=1, module=1, module_chapter=3, group=3, assistant=5, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 7, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift4')),
  Gene(lab=1, module=1, module_chapter=8, group=3, assistant=3, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 28, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift4'

#### Repair

```python

In [130]:
class Repair:
    def __init__(self, chromosome, **kwargs):
        self.chromosome = chromosome
        self.factory = kwargs.get("factory", Factory())
        self.repaired_log = []
        
    def repair(self):
        if settings.DEBUG:
            logger.info("Repairing...")

        chromosome = deepcopy(self.chromosome)
        for gene in chromosome:
            constraint_checker = ConstraintChecker(gene)
            if not constraint_checker.schedule_constraint():
                feasible_time_slot = self.find_feasible_solution(gene)
                if feasible_time_slot:
                    chromosome[chromosome.index(gene)].time_slot = feasible_time_slot
                else:
                    chromosome[chromosome.index(gene)].time_slot = self.factory.create_time_slot(gene.module)   
        return chromosome
    
    def find_feasible_solution(self, gene, max_iteration=100):
        if settings.DEBUG:
            logger.info("Finding feasible solution...")

        new_gene = deepcopy(gene)
        for _ in range(max_iteration):
            feasible_time_slot = self.factory.create_time_slot(gene.module)
            new_gene.time_slot = feasible_time_slot
            constraint_checker = ConstraintChecker(new_gene)
            if constraint_checker.schedule_constraint():
                if settings.DEBUG:
                    logger.info("Found!")
                return feasible_time_slot
            
        if settings.DEBUG:
            logger.info("No feasible solution found, returning None...")
        return None

    
#test Repair
factory = Factory()
population = factory.generate_population(1)
population.chromosome[0].genes
repair = Repair(population.chromosome[0].genes)
a = repair.repair()

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...
INFO:__main__:Repairing...
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!
INFO:__main__:Finding feasible solution...
INFO:__main__:Found!

#### Selection

```python

In [148]:
import numpy as np

class Selection:
    def __init__(self, population, **kwargs):
        self.population = population
        self.tournament_size = kwargs.get("tournament_size", 3)
        self.elitism_size = kwargs.get("elitism_size", 1)
        self.elitism_offset = kwargs.get("elitism_offset", 0)
        self.elitism_range = kwargs.get("elitism_range", 1)
        self.elitism_probability = kwargs.get("elitism_probability", 0.5)
        self.method = kwargs.get("method", "tournament")
        
    def select(self):
        if settings.DEBUG:
            logger.info("Selecting...")
        if self.method == "tournament":
            return self.tournament_selection()
        elif self.method == "elitism":
            return self.elitism_selection()
        elif self.method == "roulette":
            return self.roulette_selection()
        
    def tournament_selection(self):
        if settings.DEBUG:
            logger.info("Tournament selection...")
        population = deepcopy(self.population)
        selected_chromosomes = []
        for _ in range(len(population.chromosome)):
            selected_chromosomes.append(self.tournament_selection_helper(population))
        return selected_chromosomes
    
    def tournament_selection_helper(self, population):
        if settings.DEBUG:
            logger.info("Tournament selection helper...")
        tournament_population = Population([], None)
        tournament_population.chromosome.append(np.random.choice(population.chromosome, self.tournament_size, replace=False))
        tournament_population.chromosome.sort(key=lambda x: x.fitness, reverse=True)
        return tournament_population.chromosome[0]
    
    def elitism_selection(self):
        if settings.DEBUG:
            logger.info("Elitism selection...")
        population = deepcopy(self.population)
        selected_chromosomes = []
        for _ in range(self.elitism_size):
            selected_chromosomes.append(self.elitism_selection_helper(population))
        return selected_chromosomes
    
    def elitism_selection_helper(self, population):
        if settings.DEBUG:
            logger.info("Elitism selection helper...")
        population.chromosome.sort(key=lambda x: x.fitness, reverse=True)
        return population.chromosome[randint(self.elitism_offset, self.elitism_range)]
    
    def roulette_selection(self):
        if settings.DEBUG:
            logger.info("Roulette selection...")
        population = deepcopy(self.population)
        selected_chromosomes = []
        for _ in range(len(population.chromosome)):
            selected_chromosomes.append(self.roulette_selection_helper(population))
        return selected_chromosomes
    
    def roulette_selection_helper(self, population):
        if settings.DEBUG:
            logger.info("Roulette selection helper...")
        fitness_sum = sum([chromosome.fitness for chromosome in population.chromosome])
        fitness_probability = [chromosome.fitness / fitness_sum for chromosome in population.chromosome]
        return np.random.choice(population.chromosome, p=fitness_probability)
    
#test Selection
factory = Factory()
population = factory.generate_population(2)
population.chromosome[0].genes
population.chromosome[1].genes
selection = Selection(population, method="roulette")
a = selection.select()
    

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...
INFO:__main__:Generating chromosome...
INFO:__main__:Selecting...
INFO:__main__:Roulette selection...
INFO:__main__:Roulette selection helper...


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'